In [1]:
 
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np
import torch
import os
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
from torch.autograd import Variable
from binaryClassifier import Net






In [2]:
classes = ['typing','noTyping']

model = Net()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = torch.load('/Users/XXX/Desktop/model_cifar.pt')
# checkpoint.eval()
model.load_state_dict(checkpoint)
model.eval()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=89888, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [3]:
data_dir = '/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test'
test_transforms = transforms.Compose([transforms.Resize((224, 224)),
            transforms.ToTensor()
                                     ])
 


def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

In [6]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
  

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

# instantiate the dataset and dataloader
data_dir = '/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test'
dataset = ImageFolderWithPaths(data_dir,transform=test_transforms) # our custom dataset
dataloader = torch.utils.data.DataLoader(dataset)



In [9]:
# iterate over data

for images, labels, paths in dataloader:

    to_pil = transforms.ToPILImage()
    for i in range(len(images)):
        image = to_pil(images[i])
        index = predict_image(image)
        print(paths , ":", "predicted :",str(classes[index]))


('/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test/b/0.jpg',) : predicted : noTyping
('/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test/b/1.jpg',) : predicted : noTyping
('/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test/b/2.jpg',) : predicted : noTyping
('/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test/b/3.jpg',) : predicted : noTyping
('/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test/b/4.jpg',) : predicted : noTyping
('/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test/b/42 copy 2.jpg',) : predicted : typing
('/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test/b/42 copy.jpg',) : predicted : typing
('/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test/b/42-1 copy 2.jpg',) : predicted : typing
('/Users/XXX/Desktop/UsageTesting-Repo/video_data/6pm-video-signin-2/test/b/42-1 copy.jpg',) : predicted : typing
('/U